In [2]:
import pandas as pd
import numpy as np
#import psycopg2
import os
import glob
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd
import wfdb
import statsmodels.api as sm
import matplotlib.image as mpimg
import tkinter as tk
from matplotlib.figure import Figure
from IPython.display import display, HTML
import csv
from collections import defaultdict
from sklearn.metrics import r2_score
import matplotlib.ticker as ticker
import pickle
from sklearn.linear_model import LogisticRegression
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D, BatchNormalization
from keras.utils import np_utils
from keras.initializers import RandomNormal
from keras import backend as Ks
from keras import optimizers
from keras.optimizers import SGD,adam
from keras.models import model_from_json
import matplotlib as mpl

Using TensorFlow backend.


In [3]:
# 0 to 109 - training (inclusive)
# 110 to 139 - validation (inclusive)
#140 to 179 - test (inclusive)
# Take 1024 points of raw data
# 4 consecutive rows to store data
# RR, SpO2, HR, Labels

In [5]:
annofile = r'RDSegAnno.csv'
dfOuranno = pd.read_csv(annofile, encoding='iso-8859-1')  
N_TOT_REC = np.shape(dfOuranno)[0]
N_TRAIN_REC = 110
N_VALID_REC = 30
N_TEST_REC = 40
Pa = 0
Pb = 110
Pc = 140
Pd = 180
count = 0
TrainDf = pd.DataFrame()
for serial_id in range(Pa,Pb,1):
    print(serial_id, end=" ")
    myfn = str(serial_id)
    myfn = myfn.zfill(3)
    myfname = r'./SeriesSegData/p'+myfn+'.csv'
    SegAnnoDf = pd.read_csv(myfname, encoding='iso-8859-1')
    TOTAL_LEN = np.shape(SegAnnoDf)[0]
    st = 0
    ed = 1024 # Samples in time series
    while (ed <=TOTAL_LEN):
        RR = SegAnnoDf['RRraw'].values[st:ed]
        SpO2 = SegAnnoDf['SpO2raw'].values[st:ed]
        HR = SegAnnoDf['HRraw'].values[st:ed]
        Labels = SegAnnoDf['Label'].values[st:ed]
        TrainDf = TrainDf.append(pd.Series(RR),ignore_index=True)
        TrainDf = TrainDf.append(pd.Series(SpO2),ignore_index=True)
        TrainDf = TrainDf.append(pd.Series(HR),ignore_index=True)
        TrainDf = TrainDf.append(pd.Series(Labels),ignore_index=True)
        count += 1
        st = st + 300
        ed = ed + 300
print(count)
TrainDf.to_csv('TrainingSeg.csv',index=False)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 1707


In [12]:
print(np.shape(TrainDf))
print(1707*4)
TrainDf.head()

(6828, 1024)
6828


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,19.0,15.4,16.6,14.5,15.8,15.9,16.5,15.0,15.6,17.8,...,19.3,17.70,18.40,18.60,20.40,19.6,18.9,18.6,17.8,19.2
1,96.0,96.1,96.8,97.0,97.0,97.0,97.0,97.9,98.0,97.8,...,98.6,98.52,98.44,98.36,98.28,98.2,97.8,99.0,98.2,98.4
2,106.3,103.8,104.4,103.4,103.5,102.7,102.2,101.8,101.1,101.4,...,76.2,75.20,76.10,77.90,71.60,74.5,72.4,70.6,75.0,76.5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
4,10.0,10.0,10.0,10.0,10.0,10.0,10.5,10.2,10.6,10.0,...,10.0,10.60,11.00,11.00,10.00,10.0,10.1,11.0,12.4,17.1


In [8]:
annofile = r'RDSegAnno.csv'
dfOuranno = pd.read_csv(annofile, encoding='iso-8859-1')  
N_TOT_REC = np.shape(dfOuranno)[0]
N_TRAIN_REC = 110
N_VALID_REC = 30
N_TEST_REC = 40
Pa = 0
Pb = 110
Pc = 140
Pd = 180
count = 0
ValidDf = pd.DataFrame()
for serial_id in range(Pb,Pc,1):
    print(serial_id, end=" ")
    myfn = str(serial_id)
    myfn = myfn.zfill(3)
    myfname = r'./SeriesSegData/p'+myfn+'.csv'
    SegAnnoDf = pd.read_csv(myfname, encoding='iso-8859-1')
    TOTAL_LEN = np.shape(SegAnnoDf)[0]
    st = 0
    ed = 1024 # Samples in time series
    while (ed <=TOTAL_LEN):
        RR = SegAnnoDf['RRraw'].values[st:ed]
        SpO2 = SegAnnoDf['SpO2raw'].values[st:ed]
        HR = SegAnnoDf['HRraw'].values[st:ed]
        Labels = SegAnnoDf['Label'].values[st:ed]
        ValidDf = ValidDf.append(pd.Series(RR),ignore_index=True)
        ValidDf = ValidDf.append(pd.Series(SpO2),ignore_index=True)
        ValidDf = ValidDf.append(pd.Series(HR),ignore_index=True)
        ValidDf = ValidDf.append(pd.Series(Labels),ignore_index=True)
        count += 1
        st = st + 300
        ed = ed + 300
print(count)
ValidDf.to_csv('ValidationSeg.csv',index=False)

110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 446


In [14]:
print(np.shape(ValidDf))
print(446*4)
ValidDf.head()

(1784, 1024)
1784


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,34.999846,34.999846,34.999846,23.816810,20.583503,24.400040,28.333425,24.333270,22.383270,21.783347,...,16.833230,13.333347,13.183619,12.783503,13.249884,12.933230,12.299923,11.999962,11.483502,11.700001
1,93.000140,93.000140,93.266792,93.683332,94.166832,94.000026,93.116666,92.616723,91.370439,92.726100,...,95.716704,95.666662,95.800107,95.599939,95.716704,95.700023,95.983356,95.899952,95.816787,95.866829
2,66.061051,66.061051,66.061051,66.061051,63.599710,63.283436,67.916381,70.099458,70.050690,68.132967,...,76.866712,76.632914,73.733379,73.082902,72.882811,72.849821,72.400153,72.050172,72.182849,71.383200
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,13.199806,13.000001,12.866460,13.016693,12.899845,14.466927,22.299806,20.488406,19.066499,17.199962,...,13.283269,15.333425,13.083464,12.916538,12.683347,12.450156,13.066771,12.516927,12.566499,12.733425


In [15]:
annofile = r'RDSegAnno.csv'
dfOuranno = pd.read_csv(annofile, encoding='iso-8859-1')  
N_TOT_REC = np.shape(dfOuranno)[0]
N_TRAIN_REC = 110
N_VALID_REC = 30
N_TEST_REC = 40
Pa = 0
Pb = 110
Pc = 140
Pd = 180
count = 0
TestDf = pd.DataFrame()
for serial_id in range(Pc,Pd,1):
    print(serial_id, end=" ")
    myfn = str(serial_id)
    myfn = myfn.zfill(3)
    myfname = r'./SeriesSegData/p'+myfn+'.csv'
    SegAnnoDf = pd.read_csv(myfname, encoding='iso-8859-1')
    TOTAL_LEN = np.shape(SegAnnoDf)[0]
    st = 0
    ed = 1024 # Samples in time series
    while (ed <=TOTAL_LEN):
        RR = SegAnnoDf['RRraw'].values[st:ed]
        SpO2 = SegAnnoDf['SpO2raw'].values[st:ed]
        HR = SegAnnoDf['HRraw'].values[st:ed]
        Labels = SegAnnoDf['Label'].values[st:ed]
        TestDf = TestDf.append(pd.Series(RR),ignore_index=True)
        TestDf = TestDf.append(pd.Series(SpO2),ignore_index=True)
        TestDf = TestDf.append(pd.Series(HR),ignore_index=True)
        TestDf = TestDf.append(pd.Series(Labels),ignore_index=True)
        count += 1
        st = st + 300
        ed = ed + 300
print(count)
TestDf.to_csv('TestSeg.csv',index=False)

140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 659


In [16]:
print(np.shape(TestDf))
print(659*4)
TestDf.head()

(2636, 1024)
2636


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,12.616388,12.616388,19.999657,9.666857,30.136072,25.392653,22.866102,21.500229,15.301556,16.150034,...,17.317006,20.233806,18.233463,16.000229,16.083314,15.733349,22.383955,16.967041,15.966239,17.133211
1,98.888708,98.816761,98.733116,98.818516,98.903916,98.450011,98.633678,98.500315,99.083489,99.099867,...,99.999488,99.999488,99.999488,99.999488,99.999488,99.999488,99.999488,99.999488,99.999488,99.999488
2,66.856373,69.117575,67.764434,67.999763,68.076501,76.315563,77.566386,79.269962,72.767727,73.049099,...,86.184021,86.999998,85.682669,82.316445,82.367604,80.965863,90.148798,84.800186,81.400710,81.999263
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,15.066148,14.999428,14.999428,14.999428,14.999428,14.999428,14.999428,14.999428,14.999428,14.999428,...,21.216983,22.566491,21.916915,18.966125,17.533532,18.616159,19.000114,19.600595,19.350080,20.816663
